# 🎯 Analyse Complète Heston: Market Data → NN Calibration → IV Surfaces

Ce notebook effectue une analyse complète en 3 étapes:
1. **Téléchargement des données** de yfinance avec enregistrement CSV
2. **Calibration Heston** via réseau de neurones PyTorch (méthode streamlit app)
3. **IV Surface** calculée avec formule analytique Heston + inversion Black-Scholes en 3D


## 1. Imports et Configuration

In [24]:
from __future__ import annotations

import math
from pathlib import Path
from typing import Callable, Dict, Tuple

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
import yfinance as yf
from datetime import datetime

from heston_torch import HestonParams, carr_madan_call_torch

torch.set_default_dtype(torch.float64)
DEVICE = torch.device("cpu")
MIN_IV_MATURITY = 0.1

print("✓ Imports réussis")

✓ Imports réussis


## 2. Configuration des Paramètres

In [25]:
# Paramètres de marché
TICKER = "SPY"
RF_RATE = 0.02
DIV_YIELD = 0.0
# Paramètres de la grille pour Heston/BS
SPAN = 50.0  # S0 ± 10
STEP = 5.0
# Paramètres calibration NN
MAX_POINTS = 1000  # Nombre max de points pour calibration
MAX_ITERS = 100    # Itérations d'optimisation
LR = 5e-3          # Learning rate
# Paramètres Monte Carlo Heston
# Années à télécharger
YEARS_AHEAD = 2.5
print(f"Configuration:")
print(f"  Ticker: {TICKER}")
print(f"  Grille: S0 ± {SPAN}, step = {STEP}")
print(f"  Calibration: {MAX_ITERS} itérations, LR = {LR}")


Configuration:
  Ticker: SPY
  Grille: S0 ± 50.0, step = 5.0
  Calibration: 100 itérations, LR = 0.005


## 3. Fonctions Utilitaires

In [26]:
def fetch_spot(symbol: str) -> float:
    """Récupère le prix spot actuel."""
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="1d")
    if hist.empty:
        raise RuntimeError("Unable to retrieve spot price.")
    return float(hist["Close"].iloc[-1])


def _select_monthly_expirations(expirations, years_ahead: float = 2.5) -> list[str]:
    """Sélectionne une expiration par mois."""
    today = pd.Timestamp.utcnow().date()
    limit_date = today + pd.Timedelta(days=365 * years_ahead)
    monthly: Dict[Tuple[int, int], Tuple[pd.Timestamp, str]] = {}
    for exp in expirations:
        exp_ts = pd.Timestamp(exp)
        exp_date = exp_ts.date()
        if not (today < exp_date <= limit_date):
            continue
        key = (exp_date.year, exp_date.month)
        if key not in monthly or exp_ts < monthly[key][0]:
            monthly[key] = (exp_ts, exp)
    return [item[1] for item in sorted(monthly.values(), key=lambda x: x[0])]


def download_options(symbol: str, option_type: str, years_ahead: float = 2.5) -> pd.DataFrame:
    """Télécharge les données d'options depuis yfinance et les sauvegarde en CSV."""
    ticker = yf.Ticker(symbol)
    spot = fetch_spot(symbol)
    expirations = ticker.options
    if not expirations:
        raise RuntimeError(f"No option expirations found for {symbol}")
    
    selected = _select_monthly_expirations(expirations, years_ahead)
    rows: list[dict] = []
    now = pd.Timestamp.utcnow().tz_localize(None)
    
    print(f"Téléchargement de {len(selected)} expirations pour {option_type}s...")
    
    for expiry in selected:
        expiry_dt = pd.Timestamp(expiry)
        T = max((expiry_dt - now).total_seconds() / (365.0 * 24 * 3600), 0.0)
        T = math.floor(T * 100) / 100  # Floor à 2 décimales
        chain = ticker.option_chain(expiry)
        data = chain.calls if option_type == "call" else chain.puts
        price_col = "C_mkt" if option_type == "call" else "P_mkt"
        
        for _, row in data.iterrows():
            rows.append({
                "S0": math.floor(spot * 100) / 100,  # Floor S0 à 2 décimales
                "K": float(row["strike"]),
                "T": T,
                price_col: float(row["lastPrice"]),
                "iv_market": float(row.get("impliedVolatility", float("nan"))),
            })
    
    df = pd.DataFrame(rows)
    
    # Sauvegarder en CSV
    try:
        out_dir = Path("data")
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / f"{option_type}_options.csv"
        df.to_csv(out_path, index=False)
        print(f"✓ Données sauvegardées: {out_path}")
    except Exception as e:
        print(f"⚠ Erreur sauvegarde CSV: {e}")
    
    return df

print("✓ Fonctions utilitaires chargées")

✓ Fonctions utilitaires chargées


## 4. Fonctions de Calibration Heston (Méthode NN)

In [27]:
def prices_from_unconstrained(
    u: torch.Tensor, S0_t: torch.Tensor, K_t: torch.Tensor, T_t: torch.Tensor, r: float, q: float
) -> torch.Tensor:
    """Calcule les prix Call à partir des paramètres non contraints."""
    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    prices = []
    for S0_i, K_i, T_i in zip(S0_t, K_t, T_t):
        price_i = carr_madan_call_torch(S0_i, r, q, T_i, params, K_i)
        prices.append(price_i)
    return torch.stack(prices)


def loss(
    u: torch.Tensor, S0_t: torch.Tensor, K_t: torch.Tensor, T_t: torch.Tensor, C_mkt_t: torch.Tensor, r: float, q: float
) -> torch.Tensor:
    """Fonction de perte: MSE entre prix modèle et marché."""
    model_prices = prices_from_unconstrained(u, S0_t, K_t, T_t, r, q)
    diff = model_prices - C_mkt_t
    return 0.5 * (diff**2).mean()


def calibrate_heston_from_calls(
    calls_df: pd.DataFrame,
    r: float,
    q: float,
    max_points: int,
    max_iters: int,
    lr: float,
) -> tuple[dict[str, float], list[float]]:
    """Calibre les paramètres Heston par optimisation PyTorch."""
    df = calls_df[["S0", "K", "T", "C_mkt"]].dropna().copy()
    n_total = len(df)
    
    if n_total > max_points:
        df = df.sort_values("T")
        idx = np.linspace(0, n_total - 1, max_points, dtype=int)
        df = df.iloc[idx]
    
    df = df.reset_index(drop=True)
    print(f"Calibration sur {len(df)} points (total: {n_total})")

    S0_t = torch.tensor(df["S0"].to_numpy(), dtype=torch.float64, device=DEVICE)
    K_t = torch.tensor(df["K"].to_numpy(), dtype=torch.float64, device=DEVICE)
    T_t = torch.tensor(df["T"].to_numpy(), dtype=torch.float64, device=DEVICE)
    C_mkt_t = torch.tensor(df["C_mkt"].to_numpy(), dtype=torch.float64, device=DEVICE)

    u = torch.tensor([1.0, -3.0, -0.5, -0.5, -3.0], dtype=torch.float64, device=DEVICE, requires_grad=True)
    optimizer = torch.optim.Adam([u], lr=lr)
    history: list[float] = []

    print("Démarrage de l'optimisation...")
    for it in range(max_iters):
        optimizer.zero_grad()
        L = loss(u, S0_t, K_t, T_t, C_mkt_t, r, q)
        L.backward()
        optimizer.step()
        curr_loss = float(L.detach().cpu())
        history.append(curr_loss)
        
        if (it + 1) % 10 == 0 or it == 0 or it == max_iters - 1:
            print(f"  Iter {it+1:03d}/{max_iters} | Loss = {curr_loss:.6e}")

    with torch.no_grad():
        params_fin = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    
    calib = {
        "kappa": float(params_fin.kappa.cpu()),
        "theta": float(params_fin.theta.cpu()),
        "sigma": float(params_fin.sigma.cpu()),
        "rho": float(params_fin.rho.cpu()),
        "v0": float(params_fin.v0.cpu()),
    }
    
    print(f"\n✓ Calibration terminée! Loss finale: {history[-1]:.6e}")
    return calib, history

print("✓ Fonctions de calibration chargées")

✓ Fonctions de calibration chargées


## 5. Fonctions Black-Scholes

In [28]:
def bs_price_option(S0: float, K: float, T: float, vol: float, r: float, option_type: str) -> float:
    """Prix Black-Scholes pour call ou put."""
    if T <= 0.0 or vol <= 0.0 or S0 <= 0.0 or K <= 0.0:
        intrinsic_call = max(0.0, S0 - K * math.exp(-r * T))
        intrinsic_put = max(0.0, K * math.exp(-r * T) - S0)
        return intrinsic_call if option_type == "call" else intrinsic_put
    
    sqrt_T = math.sqrt(T)
    vol_sqrt_T = vol * sqrt_T
    d1 = (math.log(S0 / K) + (r + 0.5 * vol * vol) * T) / vol_sqrt_T
    d2 = d1 - vol_sqrt_T
    discount = math.exp(-r * T)
    
    if option_type == "call":
        return S0 * 0.5 * (1.0 + math.erf(d1 / math.sqrt(2.0))) - K * discount * 0.5 * (
            1.0 + math.erf(d2 / math.sqrt(2.0))
        )
    return K * discount * 0.5 * (1.0 + math.erf(-d2 / math.sqrt(2.0))) - S0 * 0.5 * (
        1.0 + math.erf(-d1 / math.sqrt(2.0))
    )


def implied_vol_option(price: float, S0: float, K: float, T: float, r: float, option_type: str) -> float:
    """Volatilité implicite par bissection."""
    if T <= 0.0 or price <= 0.0 or S0 <= 0.0 or K <= 0.0:
        return 0.0
    
    intrinsic = bs_price_option(S0, K, T, 0.0, r, option_type)
    if price <= intrinsic + 1e-8:
        return 0.0
    
    vol_low, vol_high = 1e-6, 1.0
    price_high = bs_price_option(S0, K, T, vol_high, r, option_type)
    
    while price_high < price and vol_high < 5.0:
        vol_high *= 2.0
        price_high = bs_price_option(S0, K, T, vol_high, r, option_type)
    
    if price_high < price:
        return float("nan")
    
    for _ in range(100):
        vol_mid = 0.5 * (vol_low + vol_high)
        price_mid = bs_price_option(S0, K, T, vol_mid, r, option_type)
        if abs(price_mid - price) < 1e-6:
            return vol_mid
        if price_mid > price:
            vol_high = vol_mid
        else:
            vol_low = vol_mid
    
    return 0.5 * (vol_low + vol_high)

print("✓ Fonctions Black-Scholes chargées")

✓ Fonctions Black-Scholes chargées


## 6. Téléchargement des Données Market


In [29]:
print(f"\n{'='*60}")
print(f"TÉLÉCHARGEMENT DES DONNÉES MARKET POUR {TICKER}")
print(f"{'='*60}\n")

calls_df = download_options(TICKER, "call", years_ahead=YEARS_AHEAD)
puts_df = download_options(TICKER, "put", years_ahead=YEARS_AHEAD)

S0 = float(calls_df["S0"].median())

print(f"\n✓ {len(calls_df)} calls et {len(puts_df)} puts téléchargés")
print(f"✓ Spot price S0 = {S0:.2f}")
print(f"✓ Plage T: [{calls_df['T'].min():.2f}, {calls_df['T'].max():.2f}] années")
print(f"✓ Plage K: [{calls_df['K'].min():.2f}, {calls_df['K'].max():.2f}]")


TÉLÉCHARGEMENT DES DONNÉES MARKET POUR SPY

Téléchargement de 14 expirations pour calls...
Téléchargement de 14 expirations pour calls...
✓ Données sauvegardées: data/call_options.csv
Téléchargement de 14 expirations pour puts...
✓ Données sauvegardées: data/call_options.csv
Téléchargement de 14 expirations pour puts...
✓ Données sauvegardées: data/put_options.csv

✓ 1815 calls et 1517 puts téléchargés
✓ Spot price S0 = 660.08
✓ Plage T: [0.00, 2.17] années
✓ Plage K: [50.00, 1340.00]
✓ Données sauvegardées: data/put_options.csv

✓ 1815 calls et 1517 puts téléchargés
✓ Spot price S0 = 660.08
✓ Plage T: [0.00, 2.17] années
✓ Plage K: [50.00, 1340.00]


## 7. Calibration des Paramètres Heston (NN)


In [30]:
print(f"\n{'='*60}")
print(f"CALIBRATION DES PARAMÈTRES HESTON VIA RÉSEAU DE NEURONES")
print(f"{'='*60}\n")

calib, history = calibrate_heston_from_calls(
    calls_df,
    r=RF_RATE,
    q=DIV_YIELD,
    max_points=MAX_POINTS,
    max_iters=MAX_ITERS,
    lr=LR,
)

print(f"\n{'='*60}")
print("PARAMÈTRES HESTON CALIBRÉS")
print(f"{'='*60}")
for key, val in calib.items():
    print(f"  {key:6s} = {val:.6f}")
print(f"{'='*60}\n")

# Plot de la convergence
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history, mode='lines', name='Loss'))
fig_loss.update_layout(
    title="Convergence de la Calibration Heston",
    xaxis_title="Itération",
    yaxis_title="Loss (MSE)",
    height=400
)
fig_loss.show()


CALIBRATION DES PARAMÈTRES HESTON VIA RÉSEAU DE NEURONES

Calibration sur 1000 points (total: 1815)
Démarrage de l'optimisation...
  Iter 001/100 | Loss = 1.041535e+02
  Iter 001/100 | Loss = 1.041535e+02
  Iter 010/100 | Loss = 1.023732e+02
  Iter 010/100 | Loss = 1.023732e+02
  Iter 020/100 | Loss = 1.006326e+02
  Iter 020/100 | Loss = 1.006326e+02
  Iter 030/100 | Loss = 9.894100e+01
  Iter 030/100 | Loss = 9.894100e+01
  Iter 040/100 | Loss = 9.727271e+01
  Iter 040/100 | Loss = 9.727271e+01
  Iter 050/100 | Loss = 9.563511e+01
  Iter 050/100 | Loss = 9.563511e+01
  Iter 060/100 | Loss = 9.403512e+01
  Iter 060/100 | Loss = 9.403512e+01
  Iter 070/100 | Loss = 9.248720e+01
  Iter 070/100 | Loss = 9.248720e+01
  Iter 080/100 | Loss = 9.100013e+01
  Iter 080/100 | Loss = 9.100013e+01
  Iter 090/100 | Loss = 8.958008e+01
  Iter 090/100 | Loss = 8.958008e+01
  Iter 100/100 | Loss = 8.823207e+01

✓ Calibration terminée! Loss finale: 8.823207e+01

PARAMÈTRES HESTON CALIBRÉS
  kappa  = 0

## 9. IV Surface (Heston analytique + Black-Scholes)


## 10. Export des Résultats


In [31]:
print(f"\n{'='*60}")
print(f"EXPORT DES RÉSULTATS")
print(f"{'='*60}\n")

# Export des prix et IV Heston en CSV
results = []
for i, T_val in enumerate(T_grid_anal):
    for j, K_val in enumerate(K_grid_anal):
        results.append({
            'K': K_val,
            'T': T_val,
            'call_price_heston': call_prices_anal[i, j],
            'put_price_heston': put_prices_anal[i, j],
            'call_iv': iv_calls[i, j],
            'put_iv': iv_puts[i, j],
        })

df_results = pd.DataFrame(results)

try:
    out_dir = Path("data")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"heston_analysis.csv"
    df_results.to_csv(out_path, index=False)
    print(f"✓ Résultats exportés: {out_path}")
except Exception as e:
    print(f"⚠ Erreur export CSV: {e}")

print(f"\n{'='*60}")
print("ANALYSE COMPLÈTE TERMINÉE!")
print(f"{'='*60}")




EXPORT DES RÉSULTATS

✓ Résultats exportés: data/heston_analysis.csv

ANALYSE COMPLÈTE TERMINÉE!


In [32]:

print(f"{'='*60}")
print(f"{'='*60}")

# Grilles demandées: K ∈ [S0-50, S0+50], T ∈ [0.1, 2.0]
K_grid_anal = np.arange(S0 - 50.0, S0 + 50.0 + STEP, STEP)
T_grid_anal = np.linspace(0.1, 2.0, 10)

# Paramètres Heston calibrés -> objet HestonParams
params_cm = HestonParams(
    kappa=torch.tensor(calib['kappa'], dtype=torch.float64),
    theta=torch.tensor(calib['theta'], dtype=torch.float64),
    sigma=torch.tensor(calib['sigma'], dtype=torch.float64),
    rho=torch.tensor(calib['rho'], dtype=torch.float64),
    v0=torch.tensor(calib['v0'], dtype=torch.float64),
)

# Pricing analytique (Carr-Madan) pour Calls + parité pour Puts
call_prices_anal = np.zeros((len(T_grid_anal), len(K_grid_anal)))
put_prices_anal = np.zeros((len(T_grid_anal), len(K_grid_anal)))
for i, T_val in enumerate(T_grid_anal):
    Ks_t = torch.tensor(K_grid_anal, dtype=torch.float64)
    call_vec = carr_madan_call_torch(S0, RF_RATE, DIV_YIELD, float(T_val), params_cm, Ks_t)
    call_prices_anal[i, :] = call_vec.detach().cpu().numpy()
    put_prices_anal[i, :] = call_prices_anal[i, :] - S0 * np.exp(-DIV_YIELD * T_val) + K_grid_anal * np.exp(-RF_RATE * T_val)

# Inversion Black-Scholes -> IV
iv_calls = np.zeros_like(call_prices_anal)
iv_puts = np.zeros_like(put_prices_anal)
for i, T_val in enumerate(T_grid_anal):
    for j, K_val in enumerate(K_grid_anal):
        iv_calls[i, j] = implied_vol_option(call_prices_anal[i, j], S0, float(K_val), float(T_val), RF_RATE, 'call')
        iv_puts[i, j] = implied_vol_option(put_prices_anal[i, j], S0, float(K_val), float(T_val), RF_RATE, 'put')

# Surface 3D
KK, TT = np.meshgrid(K_grid_anal, T_grid_anal)
fig1 = go.Figure(data=[go.Surface(x=KK, y=TT, z=iv_calls, colorscale='Viridis', colorbar=dict(title='IV'))])
fig1.update_layout(title=f"IV Surface Calls (Heston analytique) - {TICKER}", scene=dict(xaxis_title='Strike K', yaxis_title='Maturité T (années)', zaxis_title='Implied Volatility'), height=600)
fig1.show()
fig2 = go.Figure(data=[go.Surface(x=KK, y=TT, z=iv_puts, colorscale='Viridis', colorbar=dict(title='IV'))])
fig2.update_layout(title=f"IV Surface Puts (Heston analytique) - {TICKER}", scene=dict(xaxis_title='Strike K', yaxis_title='Maturité T (années)', zaxis_title='Implied Volatility'), height=600)
fig2.show()


